In [7]:
import pandas as pd
df = pd.read_csv ("data83.csv")
print(df)
# print(len(X_train_o[0,:])) #经过多项式变化的特征数，和前三项不能同时存在

     Sample  Z_1  Z_2    Z_    Z+  Nmen_1  Nmen_2  Nmen_  Nmen+   X_1  ...  \
0     Ni_Sc   28   21   3.5  24.5      61      11   25.0   36.0  1.91  ...   
1     Ni_Mn   28   25   1.5  26.5      61      52    4.5   56.5  1.91  ...   
2     Ni_Co   28   27   0.5  27.5      61      58    1.5   59.5  1.91  ...   
3     Ni_Cu   28   29   0.5  28.5      61      64    1.5   62.5  1.91  ...   
4     Ni_Nb   28   41   6.5  34.5      61      47    7.0   54.0  1.91  ...   
..      ...  ...  ...   ...   ...     ...     ...    ...    ...   ...  ...   
161  Ta_ZrE   40   73  16.5  56.5      44      48    2.0   46.0  1.33  ...   
162   Hf_YE   39   72  16.5  55.5      12      45   16.5   28.5  1.22  ...   
163   W_HfE   72   74   1.0  73.0      45      51    3.0   48.0  1.30  ...   
164  Ni_ReE   75   28  23.5  51.5      54      61    3.5   57.5  1.90  ...   
165   Cu_VE   23   29   3.0  26.0      46      64    9.0   55.0  1.63  ...   

        R+  Rp_1  Rp_2    Rp_    Rp+  Rv_1  Rv_2    Rv_    Rv+ 

In [8]:
import numpy as np
import pandas as pd
import csv
import pickle
from joblib import dump, load
from sklearn import utils
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from gplearn.genetic import SymbolicTransformer, SymbolicRegressor
# SymbolicRegressor是拟合公式使用的
import datetime
import time

In [9]:
#normalization
def scaling(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    return x

In [10]:
#read data
rawData = pd.read_excel('data83.xlsx')
# 将原始数据分割为两部分

inputData = np.array(rawData.iloc[:,1:-1])
inputData = scaling(inputData)
outputData = np.array(rawData.iloc[:,-1])
outputData

array([0.22, 0.5 , 0.42, 1.11, 0.29, 0.73, 1.21, 0.31, 0.36, 0.3 , 0.78,
       0.5 , 0.32, 0.82, 0.89, 0.49, 0.55, 0.61, 0.72, 0.5 , 0.85, 0.22,
       0.72, 0.2 , 1.  , 0.72, 0.72, 0.5 , 0.56, 0.31, 0.4 , 0.67, 0.43,
       0.52, 0.99, 0.83, 0.73, 0.39, 0.95, 0.56, 0.36, 0.38, 0.94, 0.78,
       0.17, 0.25, 0.75, 0.31, 0.33, 0.42, 0.71, 0.33, 0.87, 0.62, 0.69,
       0.75, 1.1 , 0.85, 0.61, 0.12, 0.8 , 0.29, 0.37, 0.55, 0.8 , 0.92,
       0.74, 0.17, 0.25, 1.2 , 0.24, 0.17, 0.13, 1.4 , 0.1 , 0.7 , 0.21,
       0.13, 0.16, 0.33, 0.14, 0.14, 0.09, 0.22, 0.5 , 0.42, 1.11, 0.29,
       0.73, 1.21, 0.31, 0.36, 0.3 , 0.78, 0.5 , 0.32, 0.82, 0.89, 0.49,
       0.55, 0.61, 0.72, 0.5 , 0.85, 0.22, 0.72, 0.2 , 1.  , 0.72, 0.72,
       0.5 , 0.56, 0.31, 0.4 , 0.67, 0.43, 0.52, 0.99, 0.83, 0.73, 0.39,
       0.95, 0.56, 0.36, 0.38, 0.94, 0.78, 0.17, 0.25, 0.75, 0.31, 0.33,
       0.42, 0.71, 0.33, 0.87, 0.62, 0.69, 0.75, 1.1 , 0.85, 0.61, 0.12,
       0.8 , 0.29, 0.37, 0.55, 0.8 , 0.92, 0.74, 0.

In [11]:
#initial parameter setting
# 对gplearn里面的超参数进行定义
best_fitness = float('-inf')
best_pc = 0
best_pcoef = 0
best_ps = 0
# best_pc = 0
# best_pcoef = 0
# best_ps = 0   这三个是参数设置
best_features = []
# 对最佳特征进行定义
best_gp = []
# # 对描述符进行定义
best_mean = float('inf')
function_set = ['add', 'sub', 'mul', 'div', 'abs', 'sqrt']
pc = np.arange(0.1, 0.51, 0.1)
# 对 pc、pcoef的范围进行粗筛过
# 节俭系数惩罚过于复杂的公式
pcoef =  np.arange(0.001, 0.006, 0.001)
# 由于后期要对超参数进行搜索，所以要换成矩阵的形式
pc_grid, pcoef_grid = np.meshgrid(pc, pcoef)
pc_grid = np.array(pc_grid).flatten()
pcoef_grid = np.array(pcoef_grid).flatten()
dimNum = len(pc_grid)
count = 0
    

In [12]:
#hyperparameter optimization
# 对于符号转换器而言，fitness是R2的体现
for i in range(dimNum):# 对网格参数的搜索
    pc0 = pc_grid[i]
    pcoef0 = pcoef_grid[i]
    ps_max = 1-pc0; ps_min = 0.92-pc0
    ps = np.arange(ps_min, ps_max, 0.01)
    ps = ps/3
    dimNum_ps = len(ps)
    for j in range(dimNum_ps):# 对网格参数的定义
        ps0 = ps[j]
        ph0 = ps0
        pp0 = 0.999999999-pc0-ph0-ps0
        gp = SymbolicTransformer(population_size=5000,  function_set=function_set, generations=20, n_components=6,
                        p_crossover=pc0, p_subtree_mutation=ps0,parsimony_coefficient=pcoef0, p_hoist_mutation=ph0, p_point_mutation=pp0,
                         const_range=(-1.0, 1.0), metric='pearson',tournament_size=20, verbose=1)
        print('********************************Round %d********************************' %(count+1))
        print('----------The round of pc=%.6f ps=%.6f pcoef=%.6f pp=%.6f ph=%.6f-----------'
              %(pc0, ps0, pcoef0, pp0, ph0))
        gp.fit(inputData, outputData)
        fitness = gp.run_details_['best_fitness'][-1]
#         可以直观的看到模型的运行细节
        print('The round of fitness=%.6f\n'%fitness)
        features_pd = pd.DataFrame(best_features)
        #PCC_GH_1 = abs(features_pd.corr('spearman').iloc[0,1:])
        #PCC_GH_2 = abs(features_pd.corr('spearman').iloc[1,[0,2,3,4]])
        #PCC_GH_3 = abs(features_pd.corr('spearman').iloc[2,[0,1,3,4]])
        #PCC_GH_4 = abs(features_pd.corr('spearman').iloc[3,[0,1,2,4]])
        #PCC_GH_5 = abs(features_pd.corr('spearman').iloc[4,[0,1,2,3]])
        #PCC_GH_mean = np.mean(PCC_GH_1)+np.mean(PCC_GH_2)+np.mean(PCC_GH_3)+np.mean(PCC_GH_4)+np.mean(PCC_GH_5)
        #PCC_GH_mean
        count = count+1
        if fitness > best_fitness :
            best_pc = pc0
            best_pcoef = pcoef0
            best_ps = ps0
            best_features = gp.transform(inputData)
            best_fitness = fitness
            best_gp = gp         

********************************Round 1********************************
----------The round of pc=0.100000 ps=0.273333 pcoef=0.001000 pp=0.353333 ph=0.273333-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.20         0.088161        3         0.405432              N/A     57.27s
   1     5.20         0.183738        3         0.418142              N/A     51.94s
   2     4.13         0.252524        7         0.518213              N/A     50.49s
   3     3.75         0.293442        7         0.536937              N/A     47.67s
   4     4.42         0.316821        9          0.59897              N/A     43.21s
   5     6.00         0.370097       11         0.613851              N/A     42.96s
   6     7.25         0.398406       15         0.631859              N/A   

  16    22.02         0.492199       34          0.69081              N/A     10.78s
  17    25.11         0.502508       34          0.69081              N/A      7.13s
  18    28.40          0.50601       34         0.696128              N/A      3.58s
  19    28.38         0.510297       34         0.696128              N/A      0.00s
The round of fitness=0.696128

********************************Round 5********************************
----------The round of pc=0.100000 ps=0.286667 pcoef=0.001000 pp=0.326667 ph=0.286667-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.24        0.0886971        3         0.405432              N/A      1.08m
   1     5.28         0.186057        4         0.412275              N/A     58.82s
   2     4.36         0.247171        5      

  12    10.54         0.450063       19         0.616156              N/A     24.33s
  13    12.23         0.461774       20         0.616918              N/A     19.82s
  14    14.06         0.470467       19         0.630328              N/A     16.76s
  15    11.88         0.466112       19          0.64474              N/A     13.09s
  16    15.63         0.486025       21         0.646027              N/A     10.53s
  17    17.74         0.497364       22         0.655077              N/A      6.74s
  18    17.74           0.4886       19          0.66071              N/A      3.37s
  19    17.83         0.496267       26         0.667315              N/A      0.00s
The round of fitness=0.667315

********************************Round 9********************************
----------The round of pc=0.200000 ps=0.240000 pcoef=0.001000 pp=0.320000 ph=0.240000-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- --------------

   8     8.37          0.38053       15         0.554061              N/A     35.30s
   9    10.70         0.408641       21         0.575465              N/A     32.68s
  10    12.51         0.420755       15         0.631859              N/A     29.58s
  11    14.21         0.433568       19         0.654561              N/A     27.66s
  12    16.00         0.447623       21         0.664652              N/A     24.42s
  13    17.80         0.468983       21         0.664652              N/A     20.38s
  14    16.51         0.485842       19         0.675642              N/A     17.53s
  15    17.48         0.487879       33         0.691673              N/A     14.14s
  16    18.72         0.494278       33         0.691673              N/A     10.33s
  17    19.24         0.494631       33         0.691673              N/A      6.92s
  18    19.03         0.503243       35         0.692219              N/A      3.44s
  19    20.06         0.506161       51         0.699517         

   4     5.31         0.313105       12         0.510442              N/A     47.35s
   5     7.19         0.343705       13         0.542144              N/A     45.08s
   6     9.39         0.370881       13         0.545218              N/A     42.69s
   7    11.50         0.399836       20         0.559587              N/A     40.69s
   8    12.63          0.42008        9           0.5734              N/A     36.45s
   9    13.31         0.428554       14         0.597776              N/A     32.97s
  10    15.76         0.435338       22         0.604664              N/A     30.27s
  11    16.98         0.439314       21          0.63002              N/A     27.93s
  12    13.64         0.439154       26         0.645413              N/A     23.66s
  13    15.44         0.471048       20         0.650402              N/A     20.19s
  14    16.94         0.489322       21         0.662027              N/A     16.92s
  15    17.37         0.491929       24         0.673364         

...

   0    13.06        0.0891766        3         0.418142              N/A      1.04m
   1     4.23         0.186691        4          0.42018              N/A     57.45s
   2     3.65          0.26394        5         0.473793              N/A     53.23s
   3     3.44         0.309666        7         0.501801              N/A     50.87s
   4     3.71         0.321919        7         0.501801              N/A     48.10s
   5     5.10         0.340699       10         0.534799              N/A     44.02s
   6     5.96         0.353129       10         0.534799              N/A     41.50s
   7     6.98         0.367379       14         0.538331              N/A     38.29s
   8     8.28         0.381657       16         0.541773              N/A     37.03s
   9     9.39         0.383408       14         0.538331              N/A     32.65s
  10     9.56          0.38585       14         0.536005              N/A     29.53s
  11     9.46         0.388666       14         0.536005         

   0    13.16        0.0887356        3         0.373885              N/A      1.04m
   1     4.01          0.18917        4         0.392257              N/A     56.61s
   2     3.83         0.266096       16         0.470017              N/A     56.15s
   3     3.09         0.311763       13         0.503019              N/A      1.07m
   4     4.29         0.329289       10         0.509709              N/A     51.89s
   5     5.41         0.348015       14         0.545107              N/A     46.23s
   6     6.64         0.369368       14         0.545107              N/A     41.09s
   7     7.55         0.390341       14         0.545107              N/A     39.81s
   8     9.84         0.402359       22         0.548252              N/A     35.64s
   9    11.04         0.415594       14         0.622344              N/A     33.17s
  10    10.56         0.420798       15         0.631859              N/A     29.30s
  11    10.25         0.418827       18         0.638032         

   0    12.87        0.0909545        3         0.372996              N/A      1.03m
   1     4.16          0.18975        6         0.428269              N/A     56.58s
   2     3.83         0.258423        5         0.505999              N/A     54.84s
   3     3.59         0.296198        7         0.559933              N/A     50.58s
   4     5.06         0.348637        9          0.59897              N/A     48.59s
   5     5.30         0.386911       12         0.604185              N/A     44.48s
   6     6.79         0.415828       17         0.606991              N/A     41.49s
   7     8.37         0.450927       11         0.613851              N/A     39.63s
   8     8.71         0.460351       11         0.613851              N/A     35.94s
   9     8.74         0.457123       19         0.639564              N/A     32.52s
  10     9.09         0.464598       19         0.639564              N/A     28.87s
  11    10.03         0.479846       19         0.639564         

   0    13.62        0.0881588        3         0.367412              N/A      1.03m
   1     4.02         0.188152        6         0.422083              N/A     56.88s
   2     3.55         0.265907        7         0.464673              N/A     52.60s
   3     2.97         0.305815        7         0.464673              N/A     50.00s
   4     4.30          0.31743        5         0.473793              N/A     48.53s
   5     5.24         0.327437       12         0.525073              N/A     44.63s
   6     6.48         0.341237       11          0.56366              N/A     41.25s
   7     6.24         0.361841       16          0.60772              N/A     38.45s
   8     8.57         0.375738       16          0.62328              N/A     37.47s
   9    10.22         0.384021       20         0.641537              N/A     32.98s
  10    11.94         0.403216       22         0.652896              N/A     30.10s
  11    13.61          0.41949       27         0.669753         

   0    13.25        0.0875934        3         0.373885              N/A      1.04m
   1     3.88         0.187532        5         0.430005              N/A     56.91s
   2     3.60         0.261037        5         0.431045              N/A     53.41s
   3     3.07         0.309769        5         0.473793              N/A     52.54s
   4     4.30         0.314461        7         0.501801              N/A     47.18s
   5     5.14         0.336339        9         0.579936              N/A     44.90s
   6     5.53         0.358532        9         0.579936              N/A     41.10s
   7     7.08         0.362635       13         0.615192              N/A     41.32s
   8     8.66         0.386977       18         0.618641              N/A     35.99s
   9     9.29         0.413991       15         0.632801              N/A     32.70s
  10     9.97         0.421244       19         0.639457              N/A     29.15s
  11    11.90         0.443502       25          0.64128         

In [13]:
#output result
print('The best fitness=%.3f;The best pc=%.6f;The best pcoef=%.6f;The best ps=%.6f'%(best_fitness, best_pc, best_pcoef, best_ps))
print('The best features: %s'%best_features)
print('The best formula: %s'%best_gp)

The best fitness=0.764;The best pc=0.400000;The best pcoef=0.001000;The best ps=0.196667
The best features: [[ 9.60603953e-01  4.52267239e+00  3.98080565e+00  2.97762644e+00
   6.69748370e+00  1.95377697e+00]
 [-2.54064467e+00  1.75671514e+00 -2.34698620e+00 -8.48626767e-01
   9.30240462e-02  2.30315586e-01]
 [-3.39554585e+00  2.68244436e-01  1.75877836e+00 -1.70002191e+00
  -2.10435513e+00  1.59599224e+00]
 [-1.00740714e+01 -2.43979908e+01 -9.42003048e+00 -1.23749181e+01
  -6.05537725e+00 -3.24863597e+00]
 [-2.28961402e-01  3.01252176e+00  1.30253189e+00 -2.43561364e+00
   1.32425783e+00  2.11973614e+00]
 [-2.17513139e+00 -2.37015984e+00  1.30333599e+00 -1.14812660e+00
  -2.20906237e-01  1.32115173e+00]
 [-1.06082805e+01 -2.54557946e+01 -1.20798372e+01 -1.36179022e+01
  -7.42135010e+00 -3.42481572e+00]
 [ 4.42638793e+00  1.84798308e+00  7.46842368e+00  6.39245005e+00
   7.54601711e+00  3.77221888e+00]
 [-3.44627836e+00 -1.06327206e+01 -2.12201300e+00 -3.04709214e+00
  -2.30606659e-01 

In [14]:
path = "_6D"
data = pd.DataFrame(best_features)
data.to_csv("list_Gpfeature"+path+".csv",index=False,sep=',')
demension = path.split('_')[1]
pickle.dump(best_gp, open(demension+"gp"+path+".pickle.dat", "wb"))
data = pd.DataFrame(best_features)
data.to_csv(demension+"list_Gpfeature"+path+".csv",index=False,sep=',')